In [195]:
import pandas
import pickle
import os
import pandas as pd
import nltk
import json
from datetime import datetime, timezone, date, time

In [196]:
STOCK_DATA_DIR = '/media/adv/Data/PROJECTS/CSE573-SWM/Stock Data/'
TIME_FRAMES = ['30-min', '1-hr', '4-hr', '1-day']

CURRENT_TIMEFRAME = '1-day'

In [197]:
# Train data on 1 hour

if CURRENT_TIMEFRAME == '1-day':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE1440.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON1440.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '4-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE240.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON240.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '1-hr':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE60.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON60.csv'),names=['date','time','open','high','low','close','vol'])
    
elif CURRENT_TIMEFRAME == '30-min':
    aapl_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'APPLE30.csv'),names=['date','time','open','high','low','close','vol'])
    amzn_stock_prices = pd.read_csv(os.path.join(STOCK_DATA_DIR, 'AMAZON30.csv'),names=['date','time','open','high','low','close','vol'])


In [198]:
# Label the stock price direction as -1, 1
for df in [aapl_stock_prices, amzn_stock_prices]:
    df['diff'] = (df['open'] <= df['close'])
    df['diff'] = df['diff'].apply(int)
    df['diff'].loc[df['diff']==0] = -1
    df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y.%m.%d').date())
    df['time'] = df['time'].apply(lambda x: datetime.strptime(x, '%H:%M').time())
    df['timestamp'] = df.apply(lambda x: datetime.combine(x['date'], x['time']), 1)
    df.sort_values(['timestamp'], axis=0, ascending=True, inplace=True)
    df.drop(columns=['date', 'time'], inplace=True)
    
aapl_stock_prices

,open,high,low,close,vol,diff,timestamp
0,93.159,94.194,93.028,93.460,2799,1,2014-06-30
1,93.973,94.546,93.611,93.993,4787,1,2014-07-01
2,94.344,94.515,93.540,93.902,4476,-1,2014-07-02
3,94.113,94.545,93.661,94.495,3454,1,2014-07-03
4,94.936,96.423,94.936,96.403,4638,1,2014-07-07
...,...,...,...,...,...,...,...
1138,156.250,156.490,153.650,155.920,62867,-1,2019-01-28
1139,156.470,161.620,150.980,160.540,66595,1,2019-01-29
1140,161.680,166.110,160.220,165.660,70344,1,2019-01-30
1141,166.100,168.980,164.530,166.770,62821,1,2019-01-31


In [199]:
# Load tokenized news data


with open('AaplExtractedTokens.pkl', 'rb') as f:
    tokenized_aapl_news_df = pickle.load(f)
    
with open('AmznExtractedTokens.pkl', 'rb') as f:
    tokenized_amzn_news_df = pickle.load(f)

In [202]:
# Join news data and stock direction to label news
columns = ['news_timestamp', 'stock_timestamp', 'source', 'tokens', 'label']
labelled_aapl_news = pd.DataFrame(columns=columns)
labelled_amzn_news = pd.DataFrame(columns=columns)

skipped_cnt = 0
for index, row in tokenized_aapl_news_df.iterrows():
    try:
        matching_row = aapl_stock_prices.loc[aapl_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_aapl_news.loc[len(labelled_aapl_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d apple news articles with corresponding stock chart missing" % skipped_cnt)

skipped_cnt = 0
for index, row in tokenized_amzn_news_df.iterrows():
    try:
        matching_row = amzn_stock_prices.loc[amzn_stock_prices['timestamp'] > row['timestamp']].reset_index(drop=True).loc[0]
        labelled_amzn_news.loc[len(labelled_amzn_news)] = [
            row['timestamp'],
            matching_row['timestamp'],
            row['source'],
            row['tokens'],
            matching_row['diff'],
        ]
    except:
        skipped_cnt += 1
print("Skipped %d amazon news articles with corresponding stock chart missing" % skipped_cnt)

Skipped %d news articles with corresponding stock chart missing 1234
Skipped %d news articles with corresponding stock chart missing 320


In [203]:
# Save labelled news data

with open('AmznLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_amzn_news, f)
with open('AaplLabelledNewsData.pkl', 'wb') as f:
    pickle.dump(labelled_aapl_news, f)

In [192]:
labelled_aapl_news

,news_timestamp,stock_timestamp,source,tokens,label
0,2018-05-01 21:48:00,2018-05-02,benzinga.com,"[rate, june, target, bullish, buy, record, tue...",-1
1,2018-05-04 13:58:00,2018-05-07,appleinsider.com,"[credit, host, fund, listen, ar, buy, life, re...",1
2,2018-05-10 23:11:00,2018-05-11,yahoo.com,"[review, home, push, third, platform, smart]",-1
3,2018-05-28 00:00:00,2018-05-29,normanobserver.com,"[stake, share, bank]",1
4,2018-05-28 00:00:00,2018-05-29,normanobserver.com,"[stake, also, share, own]",1
...,...,...,...,...,...
995,2018-06-07 18:48:00,2018-06-08,huronreport.com,"[quarter, cut, hold, c, stake, also, share]",1
996,2018-06-07 20:00:00,2018-06-08,thestreet.com,"[chip, tablet, buy, phone, alert, hold, use, r...",1
997,2018-06-07 21:17:00,2018-06-08,yahoo.com,"[user, model, ole, adopt, surpass, twitter, cr...",1
998,2018-06-07 21:45:00,2018-06-08,yahoo.com,"[fell, lower, earn, result, media, social, inc...",1


In [193]:
labelled_amzn_news

,news_timestamp,stock_timestamp,source,tokens,label
